In [4]:
import os
import pandas as pd

1. need to parse useful columns
2. hospitilizations/deaths targets are missing
3. compare delta vs omicron waves
    * Delta and Omicron not correlated the same in hosp, split dataset? 
    * numbers for hosp include those there for flagged for COVID, not recessarily there because of COVID
4. how to handle the metrics and normalization. 
    * metric for hosp will be change in hospitlizations? Like 10 increase? % increase
    
**NEED DATA ON:**
1. hospitilizations, vaccine rates, Omicron/Delta/Alpha proportions

## Google Data
* missing data in most columns. train quick RNN on seasonality data

In [5]:
# can also download directly from source: https://storage.googleapis.com/covid19-open-data/v3/ - needs to download though
file = 'mobility.csv'  #epidemiology.csv, mobility.csv
df = pd.read_csv(os.path.join('data', file))

In [6]:
# extract from location_key all Canadian entries
is_CA = df['location_key'].apply(lambda x: x.startswith('CA') if isinstance(x, str) else False)
df_CA = df[is_CA]

In [8]:
df_CA['location_key'].unique()

array(['CA', 'CA_AB', 'CA_BC', 'CA_MB', 'CA_NB', 'CA_NL', 'CA_NS',
       'CA_NT', 'CA_NU', 'CA_ON', 'CA_PE', 'CA_QC', 'CA_SK', 'CA_YT'],
      dtype=object)

In [9]:
df_CA

,date,location_key,mobility_retail_and_recreation,mobility_grocery_and_pharmacy,mobility_parks,mobility_transit_stations,mobility_workplaces,mobility_residential
1359856,2020-02-15,CA,4.0,2.0,10.0,3.0,1.0,0.0
1359857,2020-02-16,CA,13.0,8.0,41.0,4.0,0.0,-2.0
1359858,2020-02-17,CA,-12.0,-15.0,63.0,-28.0,-52.0,11.0
1359859,2020-02-18,CA,-1.0,4.0,6.0,-1.0,-1.0,1.0
1359860,2020-02-19,CA,1.0,1.0,9.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
1369662,2022-01-27,CA_YT,-21.0,NaN,NaN,NaN,-8.0,NaN
1369663,2022-01-28,CA_YT,-28.0,NaN,NaN,NaN,-9.0,NaN
1369664,2022-01-29,CA_YT,-30.0,NaN,NaN,NaN,-9.0,NaN
1369665,2022-01-30,CA_YT,-35.0,NaN,NaN,NaN,NaN,NaN


## Government of Canada Data
* numconf - number of confirmed cases - cumulative total, not helpful
* numdeaths - number of deaths - bad lag time
* numtoday - number of total counts from last update subtracted from total counts of current update
* numactive - number of active cases - useful!
* rateactive - number of active cases per 100k - usefull!

In [58]:
file = 'covid19-download.csv'  #epidemiology.csv, mobility.csv
df = pd.read_csv(os.path.join('data', file))